In [51]:
files = list(Path("/Users/kyledorman/data/planet_coverage/dove/results_2023").glob("*/*/*/data.parquet"))
len(files)

2041

In [52]:
pd.read_parquet(files[:1])

,has_8_channel,id,acquired,clear_percent,item_type,quality_category,satellite_azimuth,satellite_id,sun_azimuth,sun_elevation,view_angle,instrument,grid_idx
0,True,20241130_101905_49_2507,2024-11-30 10:19:05.494337+00:00,99,PSScene,test,108.0,2507,173.2,14.0,1.8,PSB.SD,202200
1,True,20241130_101905_49_2507,2024-11-30 10:19:05.494337+00:00,99,PSScene,test,108.0,2507,173.2,14.0,1.8,PSB.SD,202201
2,True,20241130_101905_49_2507,2024-11-30 10:19:05.494337+00:00,99,PSScene,test,108.0,2507,173.2,14.0,1.8,PSB.SD,202202
3,True,20241130_101905_49_2507,2024-11-30 10:19:05.494337+00:00,99,PSScene,test,108.0,2507,173.2,14.0,1.8,PSB.SD,202457
4,True,20241130_101905_49_2507,2024-11-30 10:19:05.494337+00:00,99,PSScene,test,108.0,2507,173.2,14.0,1.8,PSB.SD,202458
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59071,True,20231201_092057_88_24b0,2023-12-01 09:20:57.889564+00:00,27,PSScene,standard,282.0,24b0,158.5,10.8,5.0,PSB.SD,203469
59072,True,20231201_092057_88_24b0,2023-12-01 09:20:57.889564+00:00,27,PSScene,standard,282.0,24b0,158.5,10.8,5.0,PSB.SD,203470
59073,True,20231201_092057_88_24b0,2023-12-01 09:20:57.889564+00:00,27,PSScene,standard,282.0,24b0,158.5,10.8,5.0,PSB.SD,206565
59074,True,20231201_092057_88_24b0,2023-12-01 09:20:57.889564+00:00,27,PSScene,standard,282.0,24b0,158.5,10.8,5.0,PSB.SD,206920


In [2]:
from pathlib import Path
from itertools import cycle
from datetime import datetime

import geopandas as gpd
from rasterio.plot import show
import rasterio
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import folium
from shapely.geometry import Point, Polygon, shape
import geopandas as gpd

from src.util import create_config

In [3]:
config_file = Path("/Users/kyledorman/data/planet_coverage/skysat/config.yaml")
config = create_config(config_file)

In [10]:
gdf = gpd.read_file(config.grid_path)
gdf_wgs84 = gdf.to_crs("EPSG:4326")

gdf.head()

,geometry
0,POINT (-2453572.234 -6748515.304)
1,POINT (-2436892.977 -6748515.304)
2,POINT (-2431333.225 -6748515.304)
3,POINT (-2425773.473 -6748515.304)
4,POINT (-2420213.721 -6748515.304)


In [34]:
from shapely.geometry import box
import cartopy.io.shapereader as shpreader

# 1. Use Cartopy's Natural Earth admin_1_states_provinces shapefile
shp_path = shpreader.natural_earth(resolution='10m',
                                   category='cultural',
                                   name='admin_1_states_provinces')
states = gpd.read_file(shp_path)

# 2. Filter to California
ca = states[(states['admin'] == 'United States of America') &
            (states['name'] == 'California')]

# 3. Project to CA Albers for buffering
ca_proj = ca.to_crs("EPSG:3310")

# 4. Buffer by 20 km
buffered = ca_proj.buffer(1e5)

# 6. Convert to GeoSeries and back to WGS84
square_wgs = gpd.GeoSeries([buffered.geometry.iloc[0]], crs="EPSG:3310").to_crs("EPSG:4326")

/opt/homebrew/Caskroom/miniconda/base/envs/planet_coverage/lib/python3.11/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [37]:
intersects.sum()

999

In [43]:
# Create the base map centered on the calculated location
centroid = square_wgs.geometry[0].centroid
base_map = folium.Map(location=[centroid.y, centroid.x], zoom_start=5, width=1000, height=800)

for idx, geo in enumerate(square_wgs):
    folium.GeoJson(
        geo,
        name=str(idx),
    ).add_to(base_map)

# Add each GeoJSON file to the map
# Add polygons to the map
intersects = gdf_wgs84.geometry.intersects(square_wgs.geometry.iloc[0])
for idx, row in gdf_wgs84[intersects].iterrows():
    pt = row["geometry"]
    folium.CircleMarker(
        location=[pt.y, pt.x],
        radius=0.5,
        color="red",
        fill=True,
        fill_opacity=0.1,
        popup=str(idx),
    ).add_to(base_map)

# Display the map
base_map

# Inspect UDM extents compared to an AOI

### !!! Set the GRID_ID !!!

In [ ]:
GRID_ID = "25059125"

In [ ]:
# The results folder for a single grid
GRID_RESULTS_DIR = SAVE_PATH / GRID_ID

# Get the UDM paths for this grid
UDM_PATHS = geojson_paths(GRID_RESULTS_DIR)

# Load the grid geometry
GRID = gpd.read_file(CONFIG.grid_dir / f"{GRID_ID}.geojson")

In [ ]:
# Load the UDM GeoJSON file
geojson_file = GRID_RESULTS_DIR / "search_geometries.geojson"
gdf = gpd.read_file(geojson_file)

zoom_level = calculate_zoom_level(GRID.total_bounds)
m = folium.Map(location=(GRID.centroid.iloc[0].y, GRID.centroid.iloc[0].x), zoom_start=zoom_level - 2)

# Add AOI to the map in blue
folium.GeoJson(
    GRID,
    name="AOI",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "blue",
        "weight": 2,
        "fillOpacity": 0.9,
    },
).add_to(m)

# Plot each polygon with a different color
for _, row in gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=next(color_cycle): {
            "fillColor": color,
            "color": color,
            "weight": 2,
            "fillOpacity": 0.05,
        },
    ).add_to(m)


# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

# Inspect downloaded grid images

### Extract the intermediates for visualization

In [ ]:
extract_grid_intermediates(config_file=CONFIG_FILE, start_date=START_DATE, grid_id=GRID_ID)

## Visualize ALL Downloaded (and reprojected) UDMs
This is the list of UDMs that were considered for Asset download.

In [ ]:
udm_dir = GRID_RESULTS_DIR / "udm_cropped"
udm_paths = tif_paths(udm_dir)
num_images = len(udm_paths)

cols = min(4, num_images)
rows = num_images // cols + num_images % cols
image_size = 5

fig, axes = plt.subplots(rows, cols, figsize=(image_size * cols, image_size * rows))
if cols == rows == 1:
    axes = [axes]
else:
    axes = axes.flatten()
for ax in axes:
    ax.axis("off")

for i, (udm_path, ax) in enumerate(zip(udm_paths, axes)):
    with rasterio.open(udm_path) as src:
        udm = (src.read(1) == 1).astype(np.uint8)

    show(~udm, ax=ax, cmap="binary", title=udm_path.stem)

plt.tight_layout()

## Visualize Downloaded Images (masked to AOI)

In [ ]:
asset_dir = GRID_RESULTS_DIR / "files_asset_cropped"
udm_dir = GRID_RESULTS_DIR / "files_udm_cropped"
image_paths = tif_paths(asset_dir)
udm_paths = tif_paths(udm_dir)

cols = min(3, len(image_paths))
rows = len(image_paths) // cols + len(image_paths) % cols
image_size = 5

fig, axes = plt.subplots(rows, cols, figsize=(image_size * cols, image_size * rows))
if cols == rows == 1:
    axes = [axes]
else:
    axes = axes.flatten()
for ax in axes:
    ax.axis("off")

for i, (img_pth, udm_path, ax) in enumerate(zip(image_paths, udm_paths, axes)):
    with rasterio.open(img_pth) as src:
        img = src.read((7, 5, 3), masked=True)

    with rasterio.open(udm_path) as src:
        udm = (src.read(1) == 1).astype(np.uint8)

    img = contrast_stretch(img, p_high=97, p_low=2)
    show(img, ax=ax, title=img_pth.stem)

plt.tight_layout()

## Visualize coverage counts

In [ ]:
cols = 2
rows = 1
image_size = 5

fig, axes = plt.subplots(rows, cols, figsize=(image_size * cols, image_size * rows))
if cols == rows == 1:
    axes = [axes]
else:
    axes = axes.flatten()
for ax in axes:
    ax.axis("off")

# Create counter
with rasterio.open(udm_paths[0]) as src:
    img = (src.read(1) == 1).astype(np.uint8)
    counter = np.zeros_like(img)

for file in udm_paths:
    with rasterio.open(file) as src:
        img = (src.read(1) == 1).astype(np.uint8)
        counter += img

print("Counts min:", np.unique(counter)[1], "max:", counter.max())

show(counter, cmap="inferno", title="Counts", ax=axes[0])
_ = show(counter.clip(0, 5), cmap="inferno", title="Clipped Counts", ax=axes[1])